# Section 0

QPM: Assignment 4

DENG Yunyun <br>
LI Jiaxin <br>
SBAI Ilyas <br>
ZHOU Zhichen <br>

Note: We had to change our database obtained from assignment 3 to get the right format for the stock prices

# Section 0 (4.0)

We retrieve a dataframe ```pricesall``` of the list of the S&P500 adjusted prices that we had saved in an SQL database in the previous assignment.
We then create the dataframe ```prices``` containing only the prices for the relevant 10 stocks.

In [1]:
import pandas as pd
import numpy as np
import sqlite3

# Connecting to the database
SnP500 = sqlite3.connect(
  database="/Users/ilyas/Library/CloudStorage/OneDrive-Personnel/EDHEC/M2/Quantitative Portfolio Management/snp500.sqlite"
)
# Retrieving the price data and setting the date as our index
pricesall = (pd.read_sql_query(
  sql="SELECT * FROM SP500dailyprice",
  con=SnP500,
  parse_dates={"date"})
)
pricesall = pricesall.set_index('date')

In [2]:
# Keeping only the data for the 10 relevant stocks
tic = ['MMM','AOS','ABT','ADM','ADBE','ADP','AES','AFL','A','AKAM']
prices = pricesall[tic]
prices.head()

,MMM,AOS,ABT,ADM,ADBE,ADP,AES,AFL,A,AKAM
date,,,,,,,,,,
2000-01-03,24.772844,2.349174,9.081108,6.514026,16.274670,25.075768,25.869474,6.998001,43.757759,321.250
2000-01-04,23.788488,2.315323,8.821647,6.446173,14.909397,25.075768,24.843611,6.812305,40.415146,300.000
2000-01-05,24.477528,2.308554,8.805433,6.344393,15.204173,24.834650,25.111229,6.773205,37.908180,283.500
2000-01-06,26.446236,2.274704,9.113542,6.378315,15.328286,25.166182,25.311947,6.910042,36.464790,236.125
2000-01-07,26.971220,2.369483,9.210837,6.480096,16.072981,25.738836,25.802568,7.046870,39.503510,248.375


# Section 1

We first define the necessary data: the risk-free rate ```Rf```, the vector of annualised mean returns ```mu```, the variance-covariance matrix of annualised returns ```sigma``` and its inverse ```sigmainv```. <br>
We use the data from the last 60 days.

In [3]:
# Define and compute the necessary data
ones = np.ones((10, 1))
Rf = 0

# Getting the daily returns and computing their mean and covariance matrix (annualised)
ret = pd.DataFrame(prices.pct_change())

mu = (ret.iloc[-60:]*252).mean()
sigma = (ret.iloc[-60:]*252).cov()

# Invert of the covariance matrix
sigmainv = np.linalg.inv(sigma)

We define two functions that we will use later to get the weights of the mean-variance portfolio with a risk-free asset and of the global-minimum-variance portfolio.

In [4]:
def solutionWithRf(mu_targ, Rf, sigma, sigmainv):
    term1Numerator = (mu_targ - Rf)
    #term1Denominator = (mu - Rf*ones).T @ Vinverse @ (mu - Rf*ones)
    term1Denominator = (mu - Rf).T @ sigmainv @ (mu - Rf)
    term1 = term1Numerator/term1Denominator
    #term2 = sigmainv @ (mu - Rf*ones)
    term2 = sigmainv @ (mu - Rf)
    weightsWithRf = term1 * term2
    portfolioVolatilityWithRf = np.sqrt(weightsWithRf.T @ sigma @
    weightsWithRf)
    
    return weightsWithRf

In [5]:
def gmv(sigmainv):
    num = sigmainv @ ones
    denom = ones.T @ sigmainv @ ones
    weights = num/denom
    np.reshape(weights,(10,1))
    return weights

In [10]:
meanvar_weights60 = solutionWithRf(mu_targ, Rf, sigma, sigmainv)
gmv_weights60 = gmv(sigmainv)

print('The mean-variance portfolio weights are:',meanvar_weights60)
print('The global-minimum variance portfolio weights are:',gmv_weights60.T )

The mean-variance portfolio weights are: [ -7.44374955  -0.46524022   0.45386609  -0.12068001   3.4012277
 -14.66679945   3.95228914  24.95527003   5.47785361  -7.3073937 ]
The global-minimum variance portfolio weights are: [[ 0.06795495  0.05695817  0.27152463  0.40017181 -0.0133539  -0.0873764
  -0.1896253   0.43319369 -0.20197398  0.26252631]]


# Section 2

We initialise the matrices that will contain the weights for the two strategies at all times.<br>
We loop through the dates, and at each date compute the weights for the two strategies based on the previous 60 days of daily returns.<br>
We arbitrarily chose a mu-targ at 0.1.

In [19]:
weights_meanvar = np.zeros((len(ret)-60,10))
weights_gmv = np.zeros((len(ret)-60,10))
Rf = 0
mu_targ = 0.3

for i in range(len(ret)-60-1):
    periodret = ret.iloc[i:i+60]
    mu = periodret.mean()
    sigma = periodret.cov()
    sigmainv = np.linalg.inv(sigma)
     
    weights_meanvar[i,:] = solutionWithRf(mu_targ, Rf, sigma, sigmainv)
    
    weights_gmv[i:i+1,:] = gmv(sigmainv).T  #i:i+1 not get a shape of (10,1) and not (10,)
    

# Section 3

We compute the out-of-sample returns for both strategy at each date t by multiplying the weights obtained from the stock returns from date t-60 to date t-1 with the stock returns of date t.

In [20]:
# Mean-variance portfolio
ret_meanvar = np.dot(weights_meanvar,(ret.iloc[61:].T))
ret_meanvar = np.sum(ret_meanvar,0)

# Global minimum variance
ret_gmv = np.dot(weights_gmv,(ret.iloc[61:].T))
ret_gmv = np.sum(ret_gmv,0)

# Section 4

We compute both Sharpe returns on annualised data. <br>
The mean-variance portfolio has a higher Sharpe ratio.

In [21]:
Sharpe_meanvar = np.mean(ret_meanvar)*np.sqrt(252)/(np.std(ret_meanvar))
Sharpe_gmv = np.mean(ret_gmv)*np.sqrt(252)/(np.std(ret_gmv))
print('The Sharpe ratio for the mean-variance portfolio is:',Sharpe_meanvar,'\n')
print('The Sharpe ratio for the mean-variance portfolio is:',Sharpe_gmv)

The Sharpe ratio for the mean-variance portfolio is: 0.7459689311427389 

The Sharpe ratio for the mean-variance portfolio is: 0.711583135223626
